# NBA Data analysis

The data set used in this exercise is taken from https://www.kaggle.com/nathanlauga/nba-games

## Imports section

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

## Data set loading

Note: loading the following CSV will give a DtypeWarning because columns (6) have mixed types. This quite trivial information could be quite important in the following steps

In [ ]:
path = os.path.join(os.path.curdir,"sample_data", "games_details.csv")
df = pd.read_csv(path)

df

## Exploratory Data Analysis

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.describe().T

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.tail()

## Null values management and data wrangling

In [ ]:
df1 = df.dropna()
df1

This simple operation (done in a separate dataframe to maintain the integrity of original loaded data) show us that all rows have at least a missing value

In [ ]:
df['COMMENT'].isnull().sum()

### Exercise 

Filter data frame to obtain only rows with null comment

In [ ]:
has_null_comments = df.COMMENT.isnull()
df[has_null_comments]

### Exercise

Drop the columns that end with "_ID", possibly in one line

In [ ]:
df = df.drop([col for col in df.columns if col.endswith("_ID")], axis=1)
df

### Exercise

Find null values in MIN

In [ ]:
has_null_min = df.MIN.isnull()
df[has_null_min]

## Plotting

### Exercise

Plot a bar chart with the top 10 players that played the most matches

In [ ]:
# Counts how may rows contains the player name
# Each row represents a played game
most_played_df = df.PLAYER_NAME.value_counts(ascending=False)
# Keep only the top 10, our target
most_played_df = most_played_df[0:10]

## Statistics extraction

### Exercise

Find player that has the highest PLUS_MINUS on average (mean)

### Exercise

Find player that has the most AST (assists)

### Exercise

Who is the most accurate shooter inside the 3 that has tried at least 1000 shots

Note: FGM stands for Field Goal Made, while FGA stands for Field Goal Attempted